# Bungee Jumping Drop

Shreya Chowdhary & Gail Romer


In [3]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [4]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
N = UNITS.newton

<Unit('newton')>

We used google to find the average weight for a human, approximated the diameter of a spherical human by curling in balls and measuring ourselves, found k value for bungee cord from https://bungeejournal.academic.wlu.edu/files/2014/11/Does-a-Bungee-Cord-Behave-Like-an-Ideal-Spring.pdf

drag coefficient of a sphere: https://en.wikipedia.org/wiki/Drag_coefficient

mass bungee taken from the jump notebook (?)

In [8]:
params = Params(init_height = 220 * m,
                g = 9.8 * m/s**2,
                mass_human = 62 * kg,
                mass_object = 10 * kg,
                diameter = 0.8128 * m, # diameter of a spherical human
                rho = 1.2 * kg/m**3,   # rho of air for drag calculations
                k = 2.7289 * N/m,      # k value for an average bungee cord
                bungee_rest_length = 100 * m, # bungee cord less than half of init_height FOR NOW
                t_end = 20 * s,
               dt = 1*s,
               C_d = 0.47,
               mass_bungee = 75 * kg)

,values
init_height,220 meter
g,9.8 meter / second ** 2
mass_human,62 kilogram
mass_object,10 kilogram
diameter,0.8128 meter
rho,1.2 kilogram / meter ** 3
k,2.7289 newton / meter
bungee_rest_length,100 meter
t_end,20 second
dt,1 second


In [13]:
def make_system(params):
    init = State(t = 0 * s, 
                 h = params.init_height, 
                 v = 0 * m/s)
    area = np.pi * (params.diameter/2)**2
    mu = params.mass_bungee/(params.mass_human + params.mass_object)
    system = System(params, init = init, area = area, mu = mu)
    return system

In [89]:
system = make_system(params)
free_fall = free_fall(system)

(                 t           h          v
0.000000  0.000000  220.000000   0.000000
0.000102  0.000102  220.000000  -0.000995
0.001117  0.001117  219.999994  -0.010943
0.011268  0.011268  219.999378  -0.110426
0.112781  0.112781  219.937673  -1.105304
1.112781  1.112781  213.919476 -10.951751
2.112781  2.112781  197.959516 -21.021866
3.112781  3.112781  171.737445 -31.505754
4.112781  4.112781  134.758612 -42.554307
4.855086  4.855086  100.000000 -51.148613, sol                                  None
t_events           [[4.8550859687534675]]
nfev                                   56
njev                                    0
nlu                                     0
status                                  1
message     A termination event occurred.
success                              True
dtype: object)


AttributeError: 'tuple' object has no attribute 't'

In [37]:
def drag_force(v, system):
    unpack(system)
    return -np.sign(v) * -rho * v**2 * C_d * area / 2

In [42]:
def whip_force(system, state):
    unpack(system)
    t, h, v = state
    a = (mu * (v**2)/2)/(mu*(bungee_rest_length + h) + 2*bungee_rest_length)
    return total_mass*a

In [39]:
def spring_force(h, system):
    unpack(system)
    return k*((init_height - h) - bungee_rest_length)

In [18]:
def free_fall_event(state, t, system):
    unpack(system)
    t, h, v = state
    
    return bungee_rest_length - h

In [70]:
def slope_func_1(state, t, system):
    unpack(system)
    t, h, v = state
    
    f_drag = drag_force(v, system)
    f_whip = whip_force(system, state)
    f_grav = (total_mass)*(-g)
    
    net_force = f_drag + f_whip + f_grav
    a = net_force/total_mass
    dhdt = v
    dvdt = a 
    
    return dt, dhdt, dvdt

### Stage 1: Freefall

In [88]:
def free_fall(sys):
    system = System(sys, total_mass = sys.mass_human + sys.mass_object)
    free_fall_results = run_ode_solver(system, slope_func_1, events = free_fall_event, max_step = dt)
    
    print(free_fall_results)
    t_final = get_last_label(free_fall_results.t)
    h_final = get_last_label(free_fall_results.h)
    v_final = get_last_label(free_fall_results.v)
    free_fall_state = State(t = t_final, h = h_final, v = v_final)
    
    return free_fall_state
    #End characteristic of Stage 1 => (Bungee_rest_length - delta Y = 0)
    # return Final Height of Free Fall Stage, final V of free fall stage, final t of free fall stage

### Stage 2: Stretch

In [24]:
final_V_of_freefall = 20 * m/s
final_height_of_freefall = 100 * m
t = 20 * s

delta_Y = 220 * m - final_height_of_freefall


F_g = -(params.mass_human+params.mass_object)*params.g
F_bungee = -params.k*(delta_Y - params.bungee_rest_length) # NEED TO MAKE THIS A FUNC OF DT
# F_drag
a = -(F_bungee - F_g)/(params.mass_human+params.mass_object)
v = final_V_of_freefall + a*dt
height = final_height_of_freefall + v*dt


# EPE_of_Bungee = (1/2)*params.k*delta_Y

#End characteristic of Stage 2 => (v = 0)
# return Final Height of Stretch Stage, final V of stretch stage, final t of stretch stage, EPE of Bungee

<Quantity(163.73399999999998, 'newton')>

### Stage 3: Release
is this a necessary stage?

In [25]:
final_V_of_stretch = 0 * m/s
final_height_of_stretch = 10 * m
t = 40 * s
EPE_of_Bungee =  (1/2)*params.k*delta_Y

delta_Y = 220 * m - final_height_of_freefall

F_g = -(params.mass_human+params.mass_object)*params.g
F_bungee = -params.k*(delta_Y - params.bungee_rest_length)
a = -(F_bungee - F_g)/(params.mass_human+params.mass_object)
v = final_V_of_freefall + a*dt
height = final_height_of_freefall + v*dt

#End characteristic of Stage 2 => (v = 0)
# return Final Height of Stretch Stage, final V of stretch stage, final t of stretch stage

<Quantity(110.95802777777777, 'meter')>

### Stage 4: Return (pull)

### Stage 5: Freefly

### End Stage: Pass Height 0